In [4]:

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from cryotherm import DATA_PATH                      # packaged resource
from cryotherm.material_db import MaterialDatabase
from cryotherm.stage import Stage
from cryotherm.conduction import Conduction
from cryotherm.radiation import Radiation
from cryotherm.solver import ThermalModel

# ---------------------------------------------------------------------
# Material DB
# ---------------------------------------------------------------------
db = MaterialDatabase(DATA_PATH)



# ---------------------------------------------------------------------
#  Stages
# ---------------------------------------------------------------------
outer_shell_temp = 290  # K
middle_stage_temp = 200  # K
inner_stage_temp = 40  # K

outer = Stage("Outer Shell", outer_shell_temp, fixed=True)
middle = Stage("Middle Stage", middle_stage_temp, fixed=False)
inner = Stage("Inner Stage", inner_stage_temp, fixed=True)

# ---------------------------------------------------------------------
#  Conduction
# ---------------------------------------------------------------------

# Middle Stage Support
outer_dia = 0.5 # outer diameter in metres
inner_dia = 0.496  # inner diameter in metres
length = 0.5  # length in metres
area = np.pi * (outer_dia**2 - inner_dia**2) / 4  # cross-sectional area
material = "G10warp"  # material of the support cylinder
print("Middle Stage Support:")
print(f"\tMaterial: {material}")
print(f"\tOuter diameter: {outer_dia} m")
print(f"\tInner diameter: {inner_dia} m")
print(f"\tLength: {length} m")
print(f"\tCross-sectional area: {area:.2e} m²")
print(f"\tA/L: {area/length:.3e} m")
print(f"\tConInt: {db.get_integral(material, T1 = middle_stage_temp, T2 = outer_shell_temp, method='legacy'):.3f} W/K")

middle_stage_mech_support = Conduction(outer, middle, length=length, area=area,
                              material=material, mat_db=db, method = "legacy")
print(f"\tPower: {middle_stage_mech_support.heat_flow(outer_shell_temp, middle_stage_temp):.2e} W")

# Inner Stage Support
outer_dia = 0.005 # outer diameter in metres
inner_dia = 0.004  # inner diameter in metres
length = 0.1  # length in metres
area = np.pi * (outer_dia**2 - inner_dia**2) / 4  # cross-sectional area
material = "G10warp"  # material of the support cylinder
number = 3  # number of support rods
print("Inner Stage Support:")
print(f"\tMaterial: {material}")
print(f"\tOuter diameter: {outer_dia} m")
print(f"\tInner diameter: {inner_dia} m")
print(f"\tLength: {length} m")
print(f"\tCross-sectional area: {area:.3e} m²")
print(f"\tA/L: {area/length:.2e} m")
print(f"\tConInt: {db.get_integral(material, T1 = middle_stage_temp, T2 = inner_stage_temp, method='legacy'):.3f} W/K")

inner_stage_mech_support = Conduction(middle, inner, length=length, area=area, number=number,
                                   material=material, mat_db=db, method = "legacy")
print(f"\tPower: {inner_stage_mech_support.heat_flow(middle_stage_temp, inner_stage_temp):.2e} W")

# Cabling: Outer -> Middle
outer_dia = 0.0001  # outer diameter in metres
inner_dia = 0.0  # inner diameter in metres
length = 0.1  # length in metres
area = np.pi * (outer_dia**2 - inner_dia**2) / 4  # cross-sectional area
material = "SS304L"  # material of the support cylinder
number = 10  # number of cables
print("Cabling: Outer -> Middle")
print(f"\tMaterial: {material}")
print(f"\tOuter diameter: {outer_dia} m")
print(f"\tInner diameter: {inner_dia} m")
print(f"\tLength: {length} m")
print(f"\tCross-sectional area: {area:.3e} m²")
print(f"\tA/L: {area/length:.2e} m")
print(f"\tConInt: {db.get_integral(material, T1 = outer_shell_temp, T2 = middle_stage_temp, method='legacy'):.3f} W/K")
cabling_outer_to_middle = Conduction(outer, middle, length=length, area=area, number=number,
                     material=material, mat_db=db, method = "legacy")
print(f"\tPower: {cabling_outer_to_middle.heat_flow(outer_shell_temp, middle_stage_temp):.2e} W")

# Cabling: Middle -> Inner
outer_dia = 0.0001  # outer diameter in metres
inner_dia = 0.0  # inner diameter in metres
length = 0.1  # length in metres
area = np.pi * (outer_dia**2 - inner_dia**2) / 4  # cross-sectional area
material = "SS304L"  # material of the support cylinder
number = 10  # number of cables
print("Cabling: Middle -> Inner")
print(f"\tMaterial: {material}")
print(f"\tOuter diameter: {outer_dia} m")
print(f"\tInner diameter: {inner_dia} m")
print(f"\tLength: {length} m")
print(f"\tCross-sectional area: {area:.3e} m²")
print(f"\tA/L: {area/length:.2e} m")
print(f"\tConInt: {db.get_integral(material, T1 = middle_stage_temp, T2 = inner_stage_temp, method='legacy'):.3f} W/K")
cabling_middle_to_inner = Conduction(middle, inner, length=length, area=area, number=number,
                     material=material, mat_db=db, method = "legacy")
print(f"\tPower: {cabling_middle_to_inner.heat_flow(middle_stage_temp, inner_stage_temp):.2e} W")

# ---------------------------------------------------------------------
# Radiation
# ---------------------------------------------------------------------
# Radiation: Outer -> Middle
area_top_plate = 0.166  # m², area of the top plate
area_top_cylinder = 0.679  # m², area of the top cylinder
area_bottom_plate = 0.135  # m², area of the bottom plate
area_bottom_cylinder = 1.218  # m², area of the bottom cylinder
total_surface_area = area_top_plate + area_top_cylinder + area_bottom_plate + area_bottom_cylinder
emissivity_outer = 0.03
emissivity_inner = 0.03

radiation_outer_to_middle = Radiation(outer, middle, emissivity1=emissivity_outer, emissivity2=emissivity_inner,
                      area=total_surface_area)
print("Radiation: Outer -> Middle")
print(f"\tPower: {radiation_outer_to_middle.heat_flow(outer_shell_temp, middle_stage_temp):.2e} W")

# Radiation: Middle -> Inner
area_top_plate = 0.166  # m², area of the top plate
area_top_cylinder = 0.679  # m², area of the top cylinder
area_bottom_plate = 0.135  # m², area of the bottom plate
area_bottom_cylinder = 1.218  # m², area of the bottom cylinder
total_surface_area = area_top_plate + area_top_cylinder + area_bottom_plate + area_bottom_cylinder
emissivity_outer = 0.03
emissivity_inner = 0.03

radiation_middle_to_inner = Radiation(middle, inner, emissivity1=emissivity_outer, emissivity2=emissivity_inner,
                      area=total_surface_area)
print("Radiation: Middle -> Inner")
print(f"\tPower: {radiation_middle_to_inner.heat_flow(middle_stage_temp, inner_stage_temp):.2e} W")



# ---------------------------------------------------------------------
#  Solve
# ---------------------------------------------------------------------
model = ThermalModel(
    stages=[outer, middle, inner],
    conductors=[middle_stage_mech_support, inner_stage_mech_support, cabling_outer_to_middle, cabling_middle_to_inner],
    radiators=[radiation_outer_to_middle, radiation_middle_to_inner],
)

model.solve()                     # raises if the optimizer fails

for s in model.stages:
    print(f"{s.name:15s}:  {s.temperature:6.2f} K   netQ={s.net_heat_flow:+8.4f} W")


Middle Stage Support:
	Material: G10warp
	Outer diameter: 0.5 m
	Inner diameter: 0.496 m
	Length: 0.5 m
	Cross-sectional area: 3.13e-03 m²
	A/L: 6.258e-03 m
	ConInt: 69.032 W/K
	Power: 4.32e-01 W
Inner Stage Support:
	Material: G10warp
	Outer diameter: 0.005 m
	Inner diameter: 0.004 m
	Length: 0.1 m
	Cross-sectional area: 7.069e-06 m²
	A/L: 7.07e-05 m
	ConInt: 78.452 W/K
	Power: 1.66e-02 W
Cabling: Outer -> Middle
	Material: SS304L
	Outer diameter: 0.0001 m
	Inner diameter: 0.0 m
	Length: 0.1 m
	Cross-sectional area: 7.854e-09 m²
	A/L: 7.85e-08 m
	ConInt: 1253.239 W/K
	Power: 9.84e-04 W
Cabling: Middle -> Inner
	Material: SS304L
	Outer diameter: 0.0001 m
	Inner diameter: 0.0 m
	Length: 0.1 m
	Cross-sectional area: 7.854e-09 m²
	A/L: 7.85e-08 m
	ConInt: 1548.383 W/K
	Power: 1.22e-03 W
Radiation: Outer -> Middle
	Power: 1.04e+01 W
Radiation: Middle -> Inner
	Power: 3.03e+00 W
Outer Shell    :  290.00 K   netQ= -6.8383 W
Middle Stage   :  244.82 K   netQ= +0.0000 W
Inner Stage    :   40.0